In [ ]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

In [ ]:
from Chemprompt.data.data_loader import DataLoader
from Chemprompt.embeddings.full_descriptors_embedding import LLMModel
from Chemprompt.models.sklearn_model import ScikitLearnModel
from Chemprompt.genetic.genetic import genetic
from Chemprompt.feature_selection.descriptor_selection import descriptor_selection

In [ ]:
# Experiment Configuration
dataset_choice = "FreeSolv"
model_repo = "CohereLabs"
model_name = "aya-expanse-8b"
device = "cuda:0"

In [ ]:
# Genetic Algorithm Parameters
num_generations = 50
pop_size = 40
top_k = 5
mutation_rate = 0.05
K = 50
mode = "f_regression"

In [ ]:
# Cross-Validation Settings
start_fold = 1
end_fold = 5
n_splits = 5

In [ ]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("./result", exist_ok=True)
result_path = f"./result/{model_name}_{dataset_choice}_{current_time}_summary.txt"

In [ ]:
# Step 1. Load LLM Model
print("[INFO] Loading LLM model...")
llm_model = LLMModel(
    model_repo=f"{model_repo}/{model_name}",
    dtype="half",
    device=device
)

In [ ]:
# Step 2. Load Dataset
print(f"[INFO] Loading dataset: {dataset_choice}")
loader = DataLoader()
x, y = loader.load_dataset(dataset_choice)
y = np.array(y).reshape(-1, 1)

In [ ]:
# Step 3. Global Descriptor Filtering (One-time)
print("[INFO] Performing global descriptor filtering...")
llm_model._filter_descriptors(x)
print(f"[INFO] {len(llm_model.property_names)} descriptors retained after filtering.")

In [1]:
# Step 4. K-Fold Cross-Validation
results = []
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(kf.split(x, y), 1):
    if fold < start_fold or fold > end_fold:
        continue

    print(f"\n===== Fold {fold}/{n_splits} =====")

    # Split data
    x_train = [x[i] for i in train_idx]
    x_test = [x[i] for i in test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Step 5. Feature Selection (Train Set Only)
    print("[INFO] Running feature selection on training data...")

    # Get descriptor values directly from llm_model (SMILES only)
    X_train = llm_model.get_descriptor_values(x_train)
    X_train_df = pd.DataFrame(X_train, columns=llm_model.property_names)

    # Fit selector on training data
    selector = descriptor_selection()
    selector.fit_feature_selector(X_train_df, y_train, k=K, method=mode)

    # Map selected columns to descriptor indices
    selected_indices = selector.selected_indices
    print(f"[INFO] Selected {len(selected_indices)} features for Fold {fold}")

    print(f"unique selected_indices: {len(set(selected_indices))}, total: {len(selected_indices)}")

    # Step 6. Genetic Algorithm
    ga = genetic(
        smiles_list=x_train,
        y=y_train,
        llm_model=llm_model,
        dataset_name=f"{dataset_choice}_fold_{fold}_GA_{mode}",
        selected_indices=selected_indices,
        num_generations=num_generations,
        pop_size=pop_size,
        top_k=top_k,
        mutation_rate=mutation_rate,
        save_predictions=True
    )
    best_flag, full_flag = ga.run()

    # Step 7. LLM Embedding Generation
    X_train_emb = llm_model.get_embeddings(x_train, full_flag)
    X_test_emb = llm_model.get_embeddings(x_test, full_flag)

    # Step 8. Regression Model Training & Evaluation
    save_dir = f"./ga_result/{dataset_choice}_fold_{fold}_{current_time}"
    os.makedirs(save_dir, exist_ok=True)

    model = ScikitLearnModel("regression", save_dir=save_dir)
    model.fit_and_evaluate(data_list = X_train, train_x = X_train_emb, train_y = y_train, test_x = X_test_emb, test_y = y_test)

print("all done.")

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/HDD1/bbq9088/miniconda3/envs/Chemprompt/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


[INFO] Loading LLM model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[INFO] Loading dataset: FreeSolv
(642, 2)
[INFO] Performing global descriptor filtering...


Filtering descriptors: 100%|██████████████████████████████████████| 642/642 [00:04<00:00, 153.16it/s]


...

Ind 1: R²=0.8636 | flag=00111111110100000111001100010000100010011111101110
Ind 2: R²=0.8755 | flag=00000011010111010001100010101000110011011011111111
Ind 3: R²=0.8768 | flag=11010101001011001101101000010110110000011101000001

=== Generation 2/2 ===


Embedding flag 00000000000011000000001001011000000000001000000000000100100000001000100000010000000000
Embedding flag 11000000010101000000000010011000000000000000001100000100100010000000000010000001100000
Embedding flag 00000000000011000000001001011000000000001000000000000100100000001000100000010000000000


Ind 1: R²=0.8755 | flag=00000011010111010001100010101000110011011011111111
Ind 2: R²=0.8768 | flag=11010101001011001101101000010110110000011101000001
Ind 3: R²=0.8632 | flag=00000011010111010001100010101000110010011101000001

Best overall R²=0.8768 | flag=11010101001011001101101000010110110000011101000001


Embedding SMILES: 100%|████████████████████████████████████████████| 128/128 [00:08<00:00, 14.61it/s]


Predictions saved to ./ga_result/FreeSolv_fold_5_20251031_052757/regression_Predictions.csv
Combined results saved to ./ga_result/FreeSolv_fold_5_20251031_052757/regression_CombinedResults.csv

[RESULTS]
rmse: 0.425
r2: 0.819
pearson: 0.908
spearman: 0.919
